In [ ]:
#Librerias

import pandas as pd
import numpy as np

# **Entrega 1**

El mapa de Medellín es un archivo donde las columnas
son separadas por puntos y comas (CSV). Almacenar el mapa en una estructura de datos; por ejemplo, una matriz (de Numpy), una lista de Python, un diccionario (dataframe de Pandas) u otra.

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/calles_de_medellin_con_acoso.csv", sep=";")

# **Entrega 2**

Incluir la implementación de un algoritmo, seleccionado
de las alternativas de solución, para resolver el primer problema.

In [ ]:
class Graph:
    
    def __init__(self, data, metric='length'): #OJO: En esta linea se indica la variable para la que quiero hallar el camino más corto: Distancia o Acoso
        self.data = data
        self.metric = metric
        self.generate_nodes()
        self.generate_graph()
        
        
    def generate_nodes(self):
        if self.data is None:
            raise Exception("There's no data")

        origin_nodes = data['origin'].to_numpy()
        destination_nodes = data['destination'].to_numpy()
        all_nodes = np.concatenate([origin_nodes, destination_nodes])
        unique_nodes = np.unique(all_nodes)
        nodes = pd.DataFrame({'coordinates': unique_nodes})
        nodes['id'] = nodes.index
        nodes['id'] = nodes['id'].apply(str)
        self.nodes = nodes.set_index('coordinates')
        
    def generate_graph(self):
        self.graph = { node: {} for node in self.nodes['id'] }
        for index, row in self.data.iterrows():
            origin = self.nodes.loc[str(row['origin'])]['id']
            dest = self.nodes.loc[str(row['destination'])]['id']
            metric = row[self.metric]
            self.graph[origin][dest] = metric
    
    def calculate_distances_from(self, start):
        if start not in self.graph.keys():
            raise Exception(f"Node '{start}' does not exist")
        path = {}
        self.adj_node = {}
        queue = []
        for node in self.graph:
            path[node] = float("inf")
            self.adj_node[node] = None
            queue.append(node)
        path[start] = 0
        while queue:
            key_min = queue[0]
            min_val = path[key_min]
            for n in range(1, len(queue)):
                if path[queue[n]] < min_val:
                    key_min = queue[n]
                    min_val = path[key_min]
            cur = key_min
            queue.remove(cur)

            for i in self.graph[cur]:
                alternate = self.graph[cur][i] + path[cur]
                if path[i] > alternate:
                    path[i] = alternate
                    self.adj_node[i] = cur

    def get_path_to(self, node, show_coordinates=False):
        if node not in self.graph.keys():
            raise Exception(f"Node '{node}' does not exist")
            
        if not self.adj_node:
            raise Exception("Distances have not been calculated")
        
        if show_coordinates:
            nodes = self.nodes.copy()
            nodes['coords'] = nodes.index
            nodes = nodes.set_index('id')
            
        
        path = [nodes.loc[node][0]] if show_coordinates else [node]
        while True:
            node = self.adj_node[node]
            if node is None:
                path.reverse()
                break
            path.append(nodes.loc[node][0] if show_coordinates else node)
        return path

In [ ]:
graph = Graph(data)
graph.calculate_distances_from('2')

In [7]:
path = "→".join(graph.get_path_to("1000", show_coordinates=False))
print('The shortest path is:')
print(path)

The shortest path is:
2→1→3→4→7→13→19→25→26→30→32→50→53→63→69→66→67→107→149→248→334→363→388→401→413→432→438→444→483→497→499→516→546→548→576→585→610→648→914→1052→1090→1066→1062→1035→1013→1000
